In [1]:
data_path='/content/drive/MyDrive/NLP_Project/premchand.tsv'
model_path='/content/drive/MyDrive/NLP_Project/hindi_keras_prediction.h5'
tokenizer_path='/content/drive/MyDrive/NLP_Project/tokenizer_hindi.pickle'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install pyiwn 

In [4]:

import pandas as pd
import pickle
from numpy import array
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Embedding
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet
import pyiwn 
iwn = pyiwn.IndoWordNet(lang=pyiwn.Language.HINDI)
import numpy as np


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


2021-11-27:16:27:29,67 INFO     [helpers.py:20] Downloading IndoWordNet data of size ~31 MB...


[██████████████████████████████████████████████████]

2021-11-27:16:27:30,788 INFO     [helpers.py:43] Extracting /root/iwn_data.tar.gz into /root...


2021-11-27:16:27:32,6 INFO     [helpers.py:48] Removing temporary zip file from /root/iwn_data.tar.gz
2021-11-27:16:27:32,14 INFO     [helpers.py:51] IndoWordNet data successfully downloaded at /root/iwn_data
2021-11-27:16:27:32,19 INFO     [iwn.py:43] Loading hindi language synsets...
2021-11-27:16:27:33,220 INFO     [utils.py:157] NumExpr defaulting to 2 threads.


In [5]:
def generate_seq(model, tokenizer, max_length, seed_text, n_words):
  in_text = seed_text
  # generate a fixed number of words
  for _ in range(n_words):
    encoded = tokenizer.texts_to_sequences([in_text])[0]
    # pre-pad sequences to a fixed length
    encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')
    probs = model.predict(encoded)
    yhat = np.argsort(probs, axis=1)[:,-10:]
    # print("yhat", yhat)
    # map predicted word index to word
    out_words = []
    for i in yhat[0]:
      out_word= ""
      for word, index in tokenizer.word_index.items():
        if index == i:
          out_word = word
          break
      out_words.append(out_word)
    # append to input
    # in_text += ' ' + out_word
  return out_words

In [ ]:
model.predict("तुम ")

In [ ]:
def syn_H(word):
  synonyms=[]
  word_all_synsets=[]
  try:
    word_all_synsets=iwn.synsets(word) 
    
    for i in range(len(word_all_synsets)):
        syn=word_all_synsets[i]
        for ele in syn.lemma_names():
          synonyms.append(ele)
  except:
    pass
  return(synonyms)
  

In [ ]:
import codecs,string
def is_hindi(character):
    maxchar = max(character)
    if u'\u0900' <= maxchar <= u'\u097f':
        return True
    else:
      return False

In [ ]:
## for english
def syn_E(word):
  synonyms=[]
  for syn in wordnet.synsets(word):
    for l in syn.lemmas():
        synonyms.append(l.name())
  return(synonyms)

In [ ]:
def eval_acc(pred_y, actual_y):
  
  synonyms=[]
  if is_hindi(actual_y):
      synonyms= syn_H(actual_y)
  else:
      synonyms= syn_E(actual_y)

  if pred_y==actual_y   or   (pred_y in synonyms) :
     return 1
  else :
    return 0

##Test Dataset 

In [ ]:
#For hindi 
df = pd.read_csv(data_path, sep = '\t')
data = " ".join(df['Text'].to_list())
data= data.replace('\n', " ")
data = data[100000:200000]
data

'कर आयेगी, क्यों? अमरनाथ-जरूर आयेगी। मालती-तो जाकर मुंह धो आओ। तुम इतने नादान हो, यह मुझे मालूम न था। साड़ी देकर चले आये, अब कल वह आपको देने आयेगी! कुछ भंग तो नहीं खा गये! अमरनाथ-खैर, इसका इम्तहान कल ही हो जाएगा, अभी से क्यों बदगुमानी करती हो। तुम शाम को ज़रा देर के  लिए मेरे घर तक चली चलना। मालती-जिससे आप कहें कि यह मेरी बीवी है! अमरनाथ-मुझे क्या खबर थी कि वह मेरे घर आने के लिए तैयार हो जाएगी, नहीं तो और कोई बहाना कर देता। मालती-तो आपकी साड़ी आपको मुबारक हो, मैं नहीं जाती। अमरनाथ-मैं तो रोज तुम्हारे घर आता हूँ, तुम एक दिन के लिए भी नहीं चल सकतीं? मालती ने निष्ठुरता से कहा-अगर मौक़ा आ जाए तो तुम अपने को मेरा शौहर कहलाना पसन्द करोगे? दिल पर हाथ  रखकर कहना। अमरनाथ दिल में कट गये, बात बनाते हुए बोले-मालती, तुम मेरे साथ अन्याय कर रही हो। बुरा न मानना, मेरे व तुम्हारे बीच प्यार और मुहब्बत दिखलाने के बावजूद एक दूरी का पर्दा पड़ा था। हम दोनों एक-दूसरे की हालत को समझते थे और इस पर्दे का हटाने की कोशिश न करते थे। यह पर्दा हमारे सम्बन्धों की अनिवार्य शर्त था। हमारे बीच एक व्यापारिक समझौता-सा हो 

In [ ]:
import numpy as np
len(np.unique(data.split()))

In [ ]:
import string
data = data.translate(str.maketrans("","", string.punctuation + "।"))

In [ ]:
data

'कर आयेगी क्यों अमरनाथजरूर आयेगी मालतीतो जाकर मुंह धो आओ तुम इतने नादान हो यह मुझे मालूम न था साड़ी देकर चले आये अब कल वह आपको देने आयेगी कुछ भंग तो नहीं खा गये अमरनाथखैर इसका इम्तहान कल ही हो जाएगा अभी से क्यों बदगुमानी करती हो तुम शाम को ज़रा देर के  लिए मेरे घर तक चली चलना मालतीजिससे आप कहें कि यह मेरी बीवी है अमरनाथमुझे क्या खबर थी कि वह मेरे घर आने के लिए तैयार हो जाएगी नहीं तो और कोई बहाना कर देता मालतीतो आपकी साड़ी आपको मुबारक हो मैं नहीं जाती अमरनाथमैं तो रोज तुम्हारे घर आता हूँ तुम एक दिन के लिए भी नहीं चल सकतीं मालती ने निष्ठुरता से कहाअगर मौक़ा आ जाए तो तुम अपने को मेरा शौहर कहलाना पसन्द करोगे दिल पर हाथ  रखकर कहना अमरनाथ दिल में कट गये बात बनाते हुए बोलेमालती तुम मेरे साथ अन्याय कर रही हो बुरा न मानना मेरे व तुम्हारे बीच प्यार और मुहब्बत दिखलाने के बावजूद एक दूरी का पर्दा पड़ा था हम दोनों एकदूसरे की हालत को समझते थे और इस पर्दे का हटाने की कोशिश न करते थे यह पर्दा हमारे सम्बन्धों की अनिवार्य शर्त था हमारे बीच एक व्यापारिक समझौतासा हो गया हम दोनों उसकी गहराई में जाते हुए डरत

In [6]:
## Load Model and Tokenizer
from tensorflow import keras
model= keras.models.load_model(model_path)


tok=tokenizer_path

with open(tok, 'rb') as handle:
    tokenizer = pickle.load(handle)
# encoded = tokenizer.texts_to_sequences([data])[0]

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 5, 10)             46070     
                                                                 
 lstm (LSTM)                 (None, 5, 300)            373200    
                                                                 
 dropout (Dropout)           (None, 5, 300)            0         
                                                                 
 lstm_1 (LSTM)               (None, 300)               721200    
                                                                 
 dropout_1 (Dropout)         (None, 300)               0         
                                                                 
 dense (Dense)               (None, 4607)              1386707   
                                                                 
Total params: 2,527,177
Trainable params: 2,527,177
Non-

In [ ]:
generate_seq(model, tokenizer, 5, "कल वह आपको देने आयेगी", 1)

In [ ]:
data_t = data.split()
X = []
y = []
for i in range(len(data_t[:-6])):
  X.append(" ".join(data_t[i:i+5]))
  y.append(data_t[i+6])

In [ ]:
X = " ".join(X)
X = X.split()
import numpy as np

len(np.unique(X))

1656

In [ ]:
X = X[5000:10000]
y = y[5000:10000]

In [ ]:
## Generate X,y corresponding to test data
# retrieve vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)

sequences = list()
for i in range(4, len(encoded)):
	sequence = encoded[i-4:i+2]  ##sequence of length=6 words  
	sequences.append(sequence)
 
print('Total Sequences: %d' % len(sequences))
# pad sequences
max_length = max([len(seq) for seq in sequences])
sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')
print('Max Sequence Length: %d' % max_length)

# split into input and output elements
sequences = array(sequences)
X, y = sequences[:,:-1],sequences[:,-1]

Vocabulary Size: 4621
Total Sequences: 13
Max Sequence Length: 6


In [ ]:
phrase

'कर आयेगी, क्यों? अमरनाथ-जरूर आयेगी।'

In [ ]:
generate_seq(model, tokenizer, 5, "कल वह आपको देने आयेगी", 1)

['होती', 'तो', 'कर', 'था', 'नहीं', 'है', 'ही', 'रही', 'न', 'हो']

In [ ]:
X[:30]

In [ ]:
## print accuracy
from tqdm import tqdm
count = 0
itr = 0
predictions = []
for phrase in tqdm(X):
  pred_y = generate_seq(model, tokenizer, 5, phrase, 1)
  # predictions.append(pred_y)
  actual_y = y[itr]
  for i in pred_y:
    val = eval_acc(i, actual_y)
    if val == 1:
      count += 1
      break
  itr += 1
print("accuracy",(count*100)/len(X))

100%|██████████| 5000/5000 [16:51<00:00,  4.94it/s]

accuracy 6.08


In [ ]:
count

6

## Graph for accuracy 